# RooBinSamplingPdf and Binned Fits

**Needs ROOT 6.24**

Here we show how the [RooBinSamplingPdf](https://root.cern.ch/doc/master/classRooBinSamplingPdf.html) can be used to reduce binning artifacts in binned fits.

We start by defining a simple Gauss fit model.

In [1]:
RooRealVar x("x", "x", -10, 10);
x.setBins(20);
RooRealVar m("m", "mean",  1.5, -10, 10);
RooRealVar s("s", "sigma", 0.5, -10, 10);
RooGaussian gaus("gaus", "Gaussian distribution", x, m, s);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#0] WARNING:InputArguments -- The parameter 's' with range [-10, 10] of the RooGaussian 'gaus' exceeds the safe range of (0, inf). Advise to limit its range.


### Generate unbinned and binned data for `x`

In [2]:
RooDataSet* dataset = gaus.generate(x, 10000);
RooDataHist binnedDataset("binnedData", "BinnedData", x, *dataset);

## Plot data and model with 20 bins and check the pulls.

For the pulls, data and PDF are compared *in the bin centres*. In bins where the curvature is high, we make an error.

In [3]:
%jsroot on
TCanvas c("c", "Pull demo", 1600, 800);
c.Divide(2,2);

c.cd(1);
auto frame = x.frame();
dataset->plotOn(frame);
gaus.plotOn(frame);
frame->Draw();

c.cd(3);
auto pulls = frame->pullHist();
pulls->Draw("P");
c.Draw();

## Eliminate the bias using RooBinSamplingPdf
[RooBinSamplingPdf](https://root.cern.ch/doc/master/classRooBinSamplingPdf.html) converts a continuous into a binned distribution.
The original PDF is integrated in each of its bins.

The constructor arguments are
- Name
- Title
- Observable whose binning should be used
- PDF to be transformed into binned PDF

In [4]:
RooBinSamplingPdf gausBinned("gausBinned", "Gauss with fine sampling in each bin", x, gaus);

In [5]:
c.cd(2);
auto frame2 = x.frame();
dataset->plotOn(frame2);
gausBinned.plotOn(frame2);
frame2->Draw();

c.cd(4);
auto pulls2 = frame2->pullHist();
pulls2->Draw("P");

c.Draw();

### Fit the bin sampling PDF to binned data
We get a fit result that's consistent with expectations.

In [6]:
auto fitResult = gausBinned.fitTo(binnedDataset, RooFit::PrintLevel(-1), RooFit::Save(), RooFit::Minos());
fitResult->Print();

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization

  RooFitResult: minimized FCN value: 8554, estimated distance to minimum: 2.28162e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 MINOS=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                     m    1.5025e+00 +/-  5.77e-03
                     s    4.9729e-01 +/-  4.41e-03



### Fit the continuous PDF to binned data
Note the bias on `s`, sigma. The higher the curvature in the bins, the stronger the bias.

In [7]:
auto fitResultBiased = gaus.fitTo(binnedDataset, RooFit::PrintLevel(-1), RooFit::Save(), RooFit::Minos());
fitResultBiased->Print();

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization

  RooFitResult: minimized FCN value: 8523.28, estimated distance to minimum: 2.62779e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 MINOS=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                     m    1.5024e+00 +/-  5.67e-03
                     s    5.6742e-01 +/-  4.01e-03

